# MVA - Homework 1 - Reinforcement Learning (2022/2023)

**Name:** RAMAMBASON Jeanne

## Instructions

* The deadline is **November 10 at 11:59 pm (Paris time).**

* By doing this homework you agree to the late day policy, collaboration and misconduct rules reported on [Piazza](https://piazza.com/class/l4y5ubadwj64mb/post/6).

* **Mysterious or unsupported answers will not receive full credit**. A correct answer, unsupported by calculations, explanation, or algebraic work will receive no credit; an incorrect answer supported by substantially correct calculations and explanations might still receive partial credit.

* Answers should be provided in **English**.

# Colab setup

In [ ]:
from IPython import get_ipython

if 'google.colab' in str(get_ipython()):
  # install rlberry library
  !pip install git+https://github.com/rlberry-py/rlberry.git@mva2021#egg=rlberry[default] > /dev/null 2>&1

  # install ffmpeg-python for saving videos
  !pip install ffmpeg-python > /dev/null 2>&1

  # packages required to show video
  !pip install pyvirtualdisplay > /dev/null 2>&1
  !apt-get install -y xvfb python-opengl ffmpeg > /dev/null 2>&1

  print("Libraries installed, please restart the runtime!")


In [ ]:
# Create directory for saving videos
!mkdir videos > /dev/null 2>&1

# Initialize display and import function to show videos
import rlberry.colab_utils.display_setup
from rlberry.colab_utils.display_setup import show_video

In [ ]:
# Useful libraries
import numpy as np
import matplotlib.pyplot as plt

# Preparation

In the coding exercises, you will use a *grid-world* MDP, which is represented in Python using the interface provided by the [Gym](https://gym.openai.com/) library. The cells below show how to interact with this MDP and how to visualize it.


In [ ]:
from rlberry.envs import GridWorld

def get_env():
  """Creates an instance of a grid-world MDP."""
  env = GridWorld(
      nrows=5,
      ncols=7,
      reward_at = {(0, 6):1.0},
      walls=((0, 4), (1, 4), (2, 4), (3, 4)),
      success_probability=0.9,
      terminal_states=((0, 6),)
  )
  return env

def render_policy(env, policy=None, horizon=50):
  """Visualize a policy in an environment

  Args:
    env: GridWorld
        environment where to run the policy
    policy: np.array
        matrix mapping states to action (Ns).
        If None, runs random policy.
    horizon: int
        maximum number of timesteps in the environment.
  """
  env.enable_rendering()
  state = env.reset()                       # get initial state
  for timestep in range(horizon):
      if policy is None:
        action = env.action_space.sample()  # take random actions
      else:
        action = policy[state]
      next_state, reward, is_terminal, info = env.step(action)
      state = next_state
      if is_terminal:
        break
  # save video and clear buffer
  env.save_video('./videos/gw.mp4', framerate=5)
  env.clear_render_buffer()
  env.disable_rendering()
  # show video
  show_video('./videos/gw.mp4')


In [ ]:
# Create an environment and visualize it
env = get_env()
render_policy(env)  # visualize random policy

# The reward function and transition probabilities can be accessed through
# the R and P attributes:
print(f"Shape of the reward array = (S, A) = {env.R.shape}")
print(f"Shape of the transition array = (S, A, S) = {env.P.shape}")
print(f"Reward at (s, a) = (1, 0): {env.R[1, 0]}")
print(f"Prob[s\'=2 | s=1, a=0]: {env.P[1, 0, 2]}")
print(f"Number of states and actions: {env.Ns}, {env.Na}")

# The states in the griworld correspond to (row, col) coordinates.
# The environment provides a mapping between (row, col) and the index of
# each state:
print(f"Index of state (1, 0): {env.coord2index[(1, 0)]}")
print(f"Coordinates of state 5: {env.index2coord[5]}")

# Part 1 - Dynamic Programming

## Question 1.1

Consider a general MDP with a discount factor of $\gamma < 1$. Assume that the horizon is infinite (so there is no termination). A policy $\pi$ in this MDP
induces a value function $V^\pi$. Suppose an affine transformation is applied to the reward, what is
the new value function? Is the optimal policy preserved?



### **Answer**

Let $T \colon x \mapsto ax + b$  with $a,b \in \mathbb{R}$, the affine transformation. 

We apply the transformation $T$ to the reward $r$ thus the state value function $V^{\pi(s),T}$ for an infinite horizon problem with a discount factor  $\gamma$ < 1 and reward $ar +b$ is: 

\begin{aligned}
V^{\pi,T}(s) = & \mathbb{E}[\sum_{t=0}^{∞}\gamma^t (ar(s_t, a_t)+b)| s_0=s, a_t~~\pi(s_t)] \\
& = a~\mathbb{E}[\sum_{t=0}^{∞}\gamma^t r(s_t, a_t)| s_0=s, a_t~~\pi(s_t)] + \frac{b}{1-\gamma}\\
& = a~V^\pi(s) + \frac{b}{1-\gamma}
\end{aligned}
with : $ V^\pi(s) = \mathbb{E}[\sum_{t=0}^{∞}\gamma^t r(s_t, a_t)| s_0=s, a_t~~\pi(s_t)]  $


If  $a>0$ :  $V^{\pi,T}$ is maximal if $V^{\pi}$ is maximal so the optimal policy is preserved.


If  $a\leq0$ : the optimal policy isn't preserved because $V^{\pi,T}$ isn't maximal when $V^{\pi}$ is maximal.

## Question 1.2

Consider an infinite-horizon $\gamma$-discounted MDP. We denote by $Q^*$ the $Q$-function of the optimal policy $\pi^*$. Prove that, for any function $Q(s, a)$ (which is **not** necessarily the value function of a policy), the following inequality holds for any state $s$:

$$
V^{\pi_Q}(s) \geq V^*(s) - \frac{2}{1-\gamma}||Q^*-Q||_\infty,
$$

where $||Q^*-Q||_\infty = \max_{s, a} |Q^*(s, a) - Q(s, a)|$ and $\pi_Q(s) \in \arg\max_a Q(s, a)$. Can you use this result to show that any policy $\pi$ such that $\pi(s) \in \arg\max_a Q^*(s, a)$ is optimal?

### **Answer**

Let $s \in S$, a state, so $ Q(s, \pi_Q(s)) $ and $Q^*(s, \pi_Q(s))$ are equal to : 

$$
Q(s, \pi_Q(s)) = r(s, \pi_Q(s))+\gamma \sum_s'p(s'|s, \pi_Q(s))V^{\pi_Q}(s)=T^{\pi_Q}V^{\pi_Q}(s)\\
Q^*(s, \pi_Q(s)) = r(s, \pi_Q(s))+\gamma \sum_s'p(s'|s, \pi_Q(s))V^*(s)=T^{\pi_Q}V^*(s)
$$

Moreover :  

\begin{aligned}
V^*(s)-V^{\pi_Q}(s)
& = Q^*(s, \pi^*(s)) - Q(s, \pi_Q(s))\\
& = \left(Q^*(s, \pi^*(s)) - Q(s, \pi^*(s))\right) + \left(Q(s, \pi^*(s)) - Q^*(s, \pi_Q(s))\right) +  \left(Q^*(s, \pi_Q(s)) - Q(s, \pi_Q(s))\right) \\
\end{aligned}
By definition, $Q(s, \pi^*(s)) \leq Q(s, \pi_Q(s))$, so:
\begin{aligned}
|V^*(s)-V^{\pi_Q}(s)|
& \leq ||Q^*-Q||_∞ + |Q(s, \pi_Q(s)) - Q^*(s, \pi_Q(s))|+  |Q^*(s, \pi_Q(s)) - Q(s, \pi_Q(s))| \\
& \leq 2||Q^*-Q||_∞ + |T^{\pi_Q}V^*(s) - T^{\pi_Q}V^{\pi_Q}(s)|
\end{aligned}

Thus:

\begin{aligned}
||V^*-V^{\pi_Q}||_∞ & \leq 2||Q^*-Q||_∞ + ||T^{\pi_Q}V^* - T^{\pi_Q}V^{\pi_Q}||_∞ \\
& \leq 2||Q^*-Q||_∞ + \gamma||V^* - V^{\pi_Q}||_∞ 
\end{aligned}\\


We got : $(1-\gamma) ||V^*-V^{\pi_Q}||_∞ \leq 2||Q^*-Q||_∞$, so for all $s \in S$, we have finally: 
$$
V^{\pi_Q}(s) \geq V^*(s) - \frac{2}{1-\gamma}||Q^*-Q||_\infty.
$$
And if $Q=Q^*$ : 
$$
V^{\pi_{Q^*}}(s) \geq V^*(s)$$
Thus, any policy $\pi$ such as $\pi(s) \in \arg\max_a Q^*(s, a)$ is optimal.

## Question 1.3

In this question, you will implement and compare the policy and value iteration algorithms for a finite MDP. 

Complete the functions `policy_evaluation`, `policy_iteration` and `value_iteration` below.


Compare value iteration and policy iteration. Highlight pros and cons of each method.

### **Answer**

[pros/cons of each method + implementation below]

In [ ]:
import time

In [ ]:
def policy_evaluation(P, R, policy, gamma=0.9, tol=1e-2):
    """
    Args:
        P: np.array
            transition matrix (NsxNaxNs)
        R: np.array
            reward matrix (NsxNa)
        policy: np.array
            matrix mapping states to action (Ns)
        gamma: float
            discount factor
        tol: float
            precision of the solution
    Return:
        value_function: np.array
            The value function of the given policy
    """
    Ns, Na = R.shape
    value_function = np.zeros(Ns)
    # ====================================================
	  # YOUR IMPLEMENTATION HERE 
    #
    delta = tol + 1

    while delta > tol:
 
      
      delta = 0 
      for s in range(Ns):
        
        a = policy[s] 
        V = R[s,a]

        for s_p in range(Ns):
          V +=  gamma*P[s,a,s_p]*value_function[s_p]
        
        delta = max(delta, np.abs(V - value_function[s]))
        value_function[s] = V
      
    # ====================================================
    return value_function

In [ ]:
def policy_iteration(P, R, gamma=0.9, tol=1e-3):
    """
    Args:
        P: np.array
            transition matrix (NsxNaxNs)
        R: np.array
            reward matrix (NsxNa)
        gamma: float
            discount factor
        tol: float
            precision of the solution
    Return:
        policy: np.array
            the final policy
        V: np.array
            the value function associated to the final policy
    """
    Ns, Na = R.shape
    V_old = np.zeros(Ns)
    policy = np.ones(Ns, dtype=int)
    # ====================================================
	  # YOUR IMPLEMENTATION HERE 
    #
    V = policy_evaluation(P, R, policy, gamma=gamma, tol=tol)
    delta = tol + 1

    it = 0
    list_time_it = []

    while delta > tol:
      it+= 1
      ts = time.time()

      delta = 0
      policy_old = np.copy(policy)
      V_old = np.copy(V)

      for s in range(Ns):
        policy[s] = np.argmax(R[s,:] + gamma*P[s,:,:].dot(V))
      
      V = policy_evaluation(P, R, policy, gamma=gamma, tol=tol)
      
      tf = time.time() - ts
      list_time_it.append(tf)
      
      delta = np.max( np.abs(V - V_old))
      #delta = np.linalg.norm(V - V_old)
    # ====================================================
    return policy, V, it, list_time_it

In [ ]:
def value_iteration(P, R, gamma=0.9, tol=1e-3):
    """
    Args:
        P: np.array
            transition matrix (NsxNaxNs)
        R: np.array
            reward matrix (NsxNa)
        gamma: float
            discount factor
        tol: float
            precision of the solution
    Return:
        Q: final Q-function (at iteration n)
        greedy_policy: greedy policy wrt Qn
        Qfs: all Q-functions generated by the algorithm (for visualization)
    """
    Ns, Na = R.shape
    Q_old = np.zeros((Ns, Na))
    Q = np.zeros((Ns, Na))
    Qfs = [Q]
    # ====================================================
	  # YOUR IMPLEMENTATION HERE 
    #

    delta = tol + 1
    it = 0
    list_time_it = []
    while delta > tol : 
      it+= 1
      ts = time.time()

      Q_old = Q
      Q = R + gamma * P.dot( np.max(Q_old, axis=1))
      greedy_policy = np.argmax(Q, axis=1)
      Qfs.append(np.copy(Q))

      delta = np.max(np.abs(Q - Q_old))
      #delta = np.linalg.norm(Q - Q_old)

      tf = time.time() - ts
      list_time_it.append(tf)

    greedy_policy = np.argmax(Q, axis=1)
    # ====================================================
    return Q, greedy_policy, Qfs, it , list_time_it

### Testing your code

In [ ]:
# Parameters
tol = 1e-5
gamma = 0.99

# Environment
env = get_env()

# run value iteration to obtain Q-values
VI_Q, VI_greedypol, all_qfunctions, VI_nb_it, VI_list_time_it = value_iteration(env.P, env.R, gamma=gamma, tol=tol)

# render the policy
print("[VI]Greedy policy: ")
render_policy(env, VI_greedypol)

# compute the value function of the greedy policy using matrix inversion
# ====================================================
# YOUR IMPLEMENTATION HERE 
# compute value function of the greedy policy
#
greedy_V = policy_evaluation(env.P,env.R, VI_greedypol, gamma= gamma, tol= tol )

# ====================================================

# show the error between the computed V-functions and the final V-function
# (that should be the optimal one, if correctly implemented)
# as a function of time
final_V = all_qfunctions[-1].max(axis=1)
norms = [ np.linalg.norm(q.max(axis=1) - final_V) for q in all_qfunctions]
plt.plot(norms)
plt.xlabel('Iteration')
plt.ylabel('Error')
plt.title("Value iteration: convergence")

#### POLICY ITERATION ####
PI_policy, PI_V, PI_nb_it, PI_list_time_it = policy_iteration(env.P, env.R, gamma=gamma, tol=tol)
print("\n[PI]final policy: ")
render_policy(env, PI_policy)

## Uncomment below to check that everything is correct
assert np.allclose(PI_policy, VI_greedypol),\
     "You should check the code, the greedy policy computed by VI is not equal to the solution of PI"
assert np.allclose(PI_V, greedy_V),\
    "Since the policies are equal, even the value function should be"

plt.show()

In [ ]:
print("###### for Value Iteration Method ######")
print("Number of iterations :", VI_nb_it,"; Average time for each iteration :", np.mean(np.array(VI_list_time_it)), "s ; Total Compute time : ", np.sum(np.array(VI_list_time_it)),"s")
print("\n")
print("###### for Policy Iteration Method ######")
print("Number of iterations :", PI_nb_it,"; Average time for each iteration :", np.mean(np.array(PI_list_time_it)), "s ; Total Compute time : ", np.sum(np.array(PI_list_time_it)),"s")
print("\n")

**Response :** 
In theory : PI must perform policy evaluation on each iteration which involves solving a linear system, thus PI is slower than VI for each iteration. VI is easier to implement since it does not require the policy evaluation step. Both PI and VI are guaranted to converge but PI requires a lot less iteration to converge.

In practice : Policy Iteration (PI) requires a lot fewer steps than Value Iteration (VI) $(4<< 1147)$ but the computation time of each iteration is a lot quicker for Value Iteration than for Policy Iteraiton : 3e-5 s<< 0.6 s. PI needs less iteration beacause at each iteration, it evaluate and improve the policy whereas VI runs through all possible actions at once to find the maximum action value. But PI iteration are slower because it computes the value function at each steps, ie resolve a linear system.

# Part 2 - Tabular RL

## Question 2.1

The code below collects two datasets of transitions (containing states, actions, rewards and next states) for a discrete MDP.

For each of the datasets:

1. Estimate the transitions and rewards, $\hat{P}$ and $\hat{R}$.
2. Compute the optimal value function and the optimal policy with respect to the estimated MDP (defined by $\hat{P}$ and $\hat{R}$), which we denote by $\hat{\pi}$ and $\hat{V}$.
3. Numerically compare the performance of $\hat{\pi}$ and $\pi^\star$ (the true optimal policy), and the error between $\hat{V}$ and $V^*$ (the true optimal value function).

Which of the two data collection methods do you think is better? Why?

### **Answer**

[answer last question + implementation below]

In [ ]:
def get_random_policy_dataset(env, n_samples):
  """Get a dataset following a random policy to collect data."""
  states = []
  actions = []
  rewards = []
  next_states = []
  
  state = env.reset()
  for _ in range(n_samples):
    action = env.action_space.sample()
    next_state, reward, is_terminal, info = env.step(action)
    states.append(state)
    actions.append(action)
    rewards.append(reward)
    next_states.append(next_state)
    # update state
    state = next_state
    if is_terminal:
      state = env.reset()

  dataset = (states, actions, rewards, next_states)
  return dataset

def get_uniform_dataset(env, n_samples):
  """Get a dataset by uniformly sampling states and actions."""
  states = []
  actions = []
  rewards = []
  next_states = []
  for _ in range(n_samples):
    state = env.observation_space.sample()
    action = env.action_space.sample()
    next_state, reward, is_terminal, info = env.sample(state, action)
    states.append(state)
    actions.append(action)
    rewards.append(reward)
    next_states.append(next_state)

  dataset = (states, actions, rewards, next_states)
  return dataset


# Collect two different datasets
num_samples = 500
env = get_env()
dataset_1 = get_random_policy_dataset(env, num_samples)
dataset_2 = get_uniform_dataset(env, num_samples)


# Item 3: Estimate the MDP with the two datasets; compare the optimal value
# functions in the true and in the estimated MDPs

# ...
Ns, Na = env.R.shape

def estimate_P_and_R(dataset,Na, Ns, num_samples):
  (states, actions, rewards, next_states) = dataset

  P_est = np.zeros(((Ns,Na,Ns)))
  R_est = np.zeros((Ns,Na))


  for i in range(len(states)):
    P_est[states[i],actions[i],next_states[i]] += 1
  
  for i in range(len(rewards)):
        R_est[states[i],actions[i]] += rewards[i]

  for s in range(Ns):
    for a in range(Na):
      if P_est[s,a,:].sum()>1e-3:
        temp = P_est[s,a,:].sum()
        R_est[s,a] /= temp
        P_est[s,a,:] = P_est[s,a,:] / temp
        

  return P_est,R_est

In [ ]:
P_est_1,R_est_1 = estimate_P_and_R(dataset_1,Na, Ns, num_samples)

P_est_2,R_est_2 = estimate_P_and_R(dataset_2,Na, Ns,num_samples)

In [ ]:
# Parameters
tol = 1e-3
gamma = 0.9


In [ ]:
PI_policy_true, PI_V_true, _, _ = policy_iteration(env.P,env.R, gamma=gamma, tol=tol)

PI_policy_est_1, PI_V_est_1, _, _ = policy_iteration(P_est_1,R_est_1, gamma=gamma, tol=tol)

PI_policy_est_2, PI_V_est_2, _, _ = policy_iteration(P_est_2,R_est_2, gamma=gamma, tol=tol)

In [ ]:
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error

In [ ]:
print('MAE for dataset 1', mean_absolute_error(PI_V_true, PI_V_est_1))
print('MAE for dataset 2', mean_absolute_error(PI_V_true, PI_V_est_2))
print('\n')
print('MSE for dataset 1',mean_squared_error(PI_V_true, PI_V_est_1))
print('MSE for dataset 2',mean_squared_error(PI_V_true, PI_V_est_2))

In [ ]:
print("Optimal policy")
render_policy(env, PI_policy_true)
print("Optimal estimated policy for dataset 1")
render_policy(env, PI_policy_est_1)
print("Optimal estimated policy for dataset 2")
render_policy(env, PI_policy_est_2)


**Response :** 
The second data collection method : the uniformly sampling, is better because at each step, it chooses randomly a state and an action whereas the random policy sampling starts from the initial state and then only chooses the action randomly. Thus, the uniformly sampling allows more exploration. 

In pratice, we see that the mean absolute error and mean squared error of the value fonction estimated with dataset 2 are much better than with dataset 1.


## Question 2.2

Suppose that $\hat{P}$ and $\hat{R}$ are estimated from a dataset of exactly $N$ i.i.d. samples from **each** state-action pair. This means that, for each $(s,a)$, we have $N$ samples $\{(s_1',r_1, \dots, s_N', r_N\}$, where $s_i' \sim P(\cdot | s,a)$ and $r_i \sim R(s,a)$ for $i=1,\dots,N$, and
$$ \hat{P}(s'|s,a) = \frac{1}{N}\sum_{i=1}^N \mathbb{1}(s_i' = s'), $$
$$ \hat{R}(s,a) = \frac{1}{N}\sum_{i=1}^N r_i.$$
Suppose that $R$ is a distribution with support in $[0,1]$. Let $\hat{V}$ be the optimal value function computed in the empirical MDP (i.e., the one with transitions $\hat{P}$ and rewards $\hat{R}$). For any $\delta\in(0,1)$, derive an upper bound to the error

$$ \| \hat{V} - V^* \|_\infty $$

which holds with probability at least $1-\delta$.

**Note** Your bound should only depend on deterministic quantities like $N$, $\gamma$, $\delta$, $S$, $A$. It should *not* dependent on the actual random samples.

**Hint** The following two inequalities may be helpful.

1. **A (simplified) lemma**. For any state $\bar{s}$,

$$ |\hat{V}(\bar{s}) - V^*(\bar{s})| \leq \frac{1}{1-\gamma}\max_{s,a} \left| R(s,a) - \hat{R}(s,a) + \gamma \sum_{s'}(P(s'|s,a) - \hat{P}(s'|s,a)) V^*(s') \right|$$

2. **Hoeffding's inequality**. Let $X_1, \dots X_N$ be $N$ i.i.d. random variables bounded in the interval $[0,b]$ for some $b>0$. Let $\bar{X} = \frac{1}{N}\sum_{i=1}^N X_i$ be the empirical mean. Then, for any $\epsilon > 0$,

$$ \mathbb{P}(|\bar{X} - \mathbb{E}[\bar{X}]| > \epsilon) \leq 2e^{-\frac{2N\epsilon^2}{b^2}}.$$

### **Answer**

Let's note $X_i = r_i + \gamma \sum_{s'} \mathbb{1}_{s_i' = s'} V^*(s')$ for all $i \in  {1,..,N} $. $(X_i)_{(1\leq i \leq N)}$ are $𝑁 $ iid random variables because $s'_i$ and $r_i$ are idd variables. 

Moreover, let's note $\bar{X}_{s,a} = \frac{1}{N}\sum_{i=1}^N X_i $. 

Thus, $\bar{X}_{s,a} =  \frac{1}{N}\sum\limits_{i=1}^N r_i + \gamma \sum\limits_{s'}\frac{1}{N}\sum\limits_{i=1}^N \mathbb{1}_{s_i' = s'}V^*(s') = \hat{R}(s,a) + \gamma\sum\limits_{s'} \hat{P}(s'\mid s,a) V^*(s') $ and $\mathbb{E}[\bar{X}_{s,a}] = R(s,a) + \gamma\sum\limits_{s'} P(s'\mid s,a) V^*(s')$.

We got :  
$ \mid \bar{X}_{s,a} - \mathbb{E}[\bar{X}_{s,a}] \mid  = \mid R(s,a) - \hat{R}(s,a) + \gamma \sum_{s'}(P(s'|s,a) - \hat{P}(s'|s,a)) V^*(s')  \mid$

Thanks to the first lemma, we got for any state $\bar{s}$:  
$ \mid \hat{V}(\bar{s}) - V^*(\bar{s}) \mid \leq \frac{1}{1-\gamma} \max_{s,a} \left| R(s,a) - \hat{R}(s,a) + \gamma \sum_{s'}(P(s'|s,a) - \hat{P}(s'|s,a)) V^*(s') \right|$


Thus,
$\max_{\bar{s}} |\hat{V}(\bar{s}) - V^*(\bar{s})|  =  \| \hat{V} - V^* \|_\infty \leq \frac{1}{1-\gamma} \max_{s,a} \left| R(s,a) - \hat{R}(s,a) + \gamma \sum_{s'}(P(s'|s,a) - \hat{P}(s'|s,a)) V^*(s') \right| 
$
ie :  $\| \hat{V} - V^* \|_\infty \leq \frac{1}{1-\gamma}\max_{s,a} \mid \bar{X}_{s,a} - \mathbb{E}[\bar{X}_{s,a}] \mid  $


So for $\epsilon>0$, 

\begin{aligned}
\mathbb{P} \left( ||\hat{V} - V^*||_{\infty} \leq \epsilon \right) 
& \geq \mathbb{P} \left( \frac{1}{1-\gamma}\max_{s,a} \mid \bar{X}_{s,a} - \mathbb{E}[\bar{X}_{s,a}] \mid \leq \epsilon \right)\\
& = \prod\limits_{s,a} \mathbb{P} \left( \frac{1}{1-\gamma} \mid \bar{X}_{s,a} - \mathbb{E}[\bar{X}_{s,a}] \mid \leq \epsilon \right) ~~~~~~~~\textrm{← because samples are independant}\\
& = \prod\limits_{s,a} \mathbb{P} \left( \mid \bar{X}_{s,a} - \mathbb{E}[\bar{X}_{s,a}] \mid \leq \epsilon (1-\gamma) \right)\\
\end{aligned}

Moreover, $(X_i)_{(1\leq i \leq N)}$ are bounded in the interval $[0, \frac{1}{1-\gamma} ]$: 
\begin{aligned}
\mid X_i \mid & \leq \mid r_i| +  \gamma \sum\limits_{s'} \mathbb{1}_{s_i' = s'}|V^*(s)\mid \\
&\leq 1 +  \gamma \sum\limits_{s'}|V^*(s')| \textrm{←  because $R$  is a distribution with support in [0,1]} \\
&\leq 1 +  \gamma \sum\limits_{s'} \mathbb{E}\left[\sum\limits_{t=0}^{\infty} \gamma^t |R(s_t,a_t)| \mid s_0 = s'\right] \\
&\leq 1 +  \gamma \sum\limits_{t=0}^{\infty} \gamma^t\\
& = 1 + \frac{\gamma}{1-\gamma} \\
& =\frac{1}{1-\gamma}
\end{aligned}

So, we can apply the Hoeffding's inequality on all $\bar{X}_{s,a}$ with $b = \frac{1}{1-\gamma}$: 
\begin{aligned}
\mathbb{P} \left( ||\hat{V} - V^*||_{\infty} \leq \epsilon \right) 
& \geq \prod\limits_{s,a} \mathbb{P} \left( \mid \bar{X}_{s,a} - \mathbb{E}[\bar{X}_{s,a}] \mid \leq \epsilon (1-\gamma) \right)\\
& \geq (1 - 2e^{-2N \epsilon^2(1-\gamma)^4})^{N_s N_a}
\end{aligned}
with $N_s = |S|$ and $N_a = |A|$.


We choose $\epsilon$ such as $(1 - 2e^{-2N \epsilon^2(1-\gamma)^4})^{N_s N_a} = 1-\delta$. 


Thus with $\epsilon = \sqrt{- \frac{\ln \left(\frac{1 - (1 - \delta)^{\frac{1}{N_s N_a}}}{2}\right)}{2(1-\gamma)^4 N}}$, we got $\mathbb{P} \left( ||\hat{V} - V^*||_{\infty} \leq \epsilon \right) \geq 1 -\delta$.

## Question 2.3

Suppose once again that we are given a dataset of $N$ samples in the form of tuples $(s_i,a_i,s_i',r_i)$. We know that each tuple contains a valid transition from the true MDP, i.e., $s_i' \sim P(\cdot | s_i, a_i)$ and $r_i \sim R(s_i,a_i)$, while the state-action pairs $(s_i,a_i)$ from which the transition started can be arbitrary.

Suppose we want to apply Q-learning to this MDP. Can you think of a way to leverage this offline data to improve the sample-efficiency of the algorithm? What if we were using SARSA instead?

### **Answer**

$\rightarrow$ Q learning is an off-policy learning algorithm, ie it can use any behavioral policy. 
To improve the sampling efficiency, we use the implementation of the Tabular Dyna-Q algorithm, the exploration policy used is the $\epsilon$-greedy policy derived from $Q$ estimated, so that we are closer to the real policy. 


$\rightarrow$ SARSA is an on-policy learning algorithm, ie it uses the value candidate $\hat{Q}$ to define the policy used to select the action. To improve the sampling efficiency of the algorithm, we can use an Importance sampling method which uses a weighted distribution. This weighted distribution favours "important" samples ie samples close to the target distribution. 

# Part 3 - RL with Function Approximation

## Question 3.1

Given a datset $(s_i, a_i, r_i, s_i')$ of (states, actions, rewards, next states), the Fitted Q-Iteration (FQI) algorithm proceeds as follows:


* We start from a $Q$ function $Q_0 \in \mathcal{F}$, where $\mathcal{F}$ is a function space;
* At every iteration $k$, we compute $Q_{k+1}$ as:

$$
Q_{k+1}\in\arg\min_{f\in\mathcal{F}} \frac{1}{2}\sum_{i=1}^N
\left(
  f(s_i, a_i) - y_i^k
\right)^2 + \lambda \Omega(f)
$$
where $y_i^k = r_i + \gamma \max_{a'}Q_k(s_i', a')$, $\Omega(f)$ is a regularization term and $\lambda > 0$ is the regularization coefficient.


Consider FQI with *linear* function approximation. That is, for a given feature map $\phi : S \rightarrow \mathbb{R}^d$, we consider a parametric family of $Q$ functions $Q_\theta(s,a) = \phi(s)^T\theta_a$ for $\theta_a\in\mathbb{R}^d$. Suppose we are applying FQI on a given dataset of $N$ tuples of the form $(s_i, a_i, r_i, s_i')$ and we are at the $k$-th iteration. Let $\theta_k \in\mathbb{R}^{d \times A}$ be our current parameter. Derive the *closed-form* update to find $\theta_{k+1}$, using $\frac{1}{2}\sum_a ||\theta_a||_2^2$ as regularization.

### **Answer**

At each iteration $k$, we compute $Q_{k+1}$ as: 
$$
Q_{k+1}\in\arg\min_{\theta\in\mathbb{R}^{d \times A}} \frac{1}{2}\sum_{i=1}^N
\left(
  \phi(s)^T\theta_{a_i} - y_i^k
\right)^2 + \frac{\lambda }{2}\sum_a ||\theta_a||_2^2
$$

ie : $$
Q_{k+1}\in\arg\min_{\theta\in\mathbb{R}^{d \times A}} F(\theta)
$$
with 
\begin{aligned}
F(\theta) 
&=  \frac{1}{2}\sum_{i=1}^N \left( \phi(s)^T\theta_{a_i} - y_i^k \right)^2 + \frac{\lambda }{2}\sum_a ||\theta_a||_2^2 \\
\end{aligned}

The minimum of $F(\theta) $ : $\theta_{k+1}$, satisfies $\nabla_{\theta}F(\theta_{k+1}) = 0$. Let's compute $\theta_{k+1}$ such as for all $a \in A$, $\nabla_{\theta_a}F(\theta_{k+1}) = 0$ thus $\nabla_{\theta}F(\theta_{k+1}) = 0$.

For $a \in A$, 
$$\nabla_{\theta_a}F(\theta_{k+1})  = \sum\limits_{\substack{i=0 \\ s.t.~a_i = a}}^N \left( (\phi(s_i)^T \theta_{a_i} - y_i^k )\phi(s_i) + \lambda \theta_{a_i} \right) = 0 $$
$\iff \theta_{k+1,a} = \left(\sum\limits_{\substack{i=0 \\ s.t. a_i = a}}^N \phi(s_i)\phi(s_i)^T + \lambda I_d \right)^{-1} \sum\limits_{\substack{i=0 \\ s.t.~a_i = a}}^N y_i^k \phi(s_i)$

And, $\theta_{k+1} = (\theta_{k+1,a})_{a \in A}$

## Question 3.2

The code below creates a larger gridworld (with more states than the one used in the previous questions), and defines a feature map. Implement linear FQI to this environment (in the function `linear_fqi()` below), and compare the approximated $Q$ function to the optimal $Q$ function computed with value iteration.

Can you improve the feature map in order to reduce the approximation error?

### **Answer**

[explanation about how you tried to reduce the approximation error + FQI implementation below]

In [ ]:
def get_large_gridworld():
  """Creates an instance of a grid-world MDP with more states."""
  walls = [(ii, 10) for ii in range(15) if (ii != 7 and ii != 8)]
  env = GridWorld(
      nrows=15,
      ncols=15,
      reward_at = {(14, 14):1.0},
      walls=tuple(walls),
      success_probability=0.9,
      terminal_states=((14, 14),)
  )
  return env


class GridWorldFeatureMap:
  """Create features for state-action pairs
  
  Args:
    dim: int
      Feature dimension
    sigma: float
      RBF kernel bandwidth
  """
  def __init__(self, env, dim=15, sigma=0.25):
    self.index2coord = env.index2coord
    self.n_states = env.Ns
    self.n_actions = env.Na
    self.dim = dim
    self.sigma = sigma

    n_rows = env.nrows
    n_cols = env.ncols

    # build similarity matrix
    sim_matrix = np.zeros((self.n_states, self.n_states))
    for ii in range(self.n_states):
        row_ii, col_ii = self.index2coord[ii]
        x_ii = row_ii / n_rows
        y_ii = col_ii / n_cols
        for jj in range(self.n_states):
            row_jj, col_jj = self.index2coord[jj]
            x_jj = row_jj / n_rows
            y_jj = col_jj / n_cols
            dist = np.sqrt((x_jj - x_ii) ** 2.0 + (y_jj - y_ii) ** 2.0)
            sim_matrix[ii, jj] = np.exp(-(dist / sigma) ** 2.0)

    # factorize similarity matrix to obtain features
    uu, ss, vh = np.linalg.svd(sim_matrix, hermitian=True)
    self.feats = vh[:dim, :]

  def map(self, observation):
    feat = self.feats[:, observation].copy()
    return feat

In [ ]:
env = get_large_gridworld()
feat_map = GridWorldFeatureMap(env)

# Visualize large gridworld
render_policy(env)

# The features have dimension (feature_dim).
feature_example = feat_map.map(1) # feature representation of s=1
print(feature_example)

# Initial vector theta representing the Q function
theta = np.zeros((feat_map.dim, env.action_space.n))
print(theta.shape)
print(feature_example @ theta) # approximation of Q(s=1, a)

In [ ]:
def linear_fqi(env, feat_map, num_iterations, lambd=0.1, gamma=0.95):
  """
  # Linear FQI implementation
  # TO BE COMPLETED
  """
  
  n_samples = 5000
  # get a dataset
  dataset = get_uniform_dataset(env, n_samples= n_samples )
  # dataset = get_random_policy_dataset(env, n_samples= n_samples )

  (states, actions, rewards, next_states) = dataset

  R_max = max(rewards) 

  theta_old = np.zeros((feat_map.dim, env.Na))
  theta_new = np.zeros((feat_map.dim, env.Na))

  for it in range(num_iterations):
    if it%10 == 0:
      print('it n°',it)

    theta_old = np.copy(theta_new)

    ## calcul de y_k
    y_k = np.zeros(n_samples)
    for i in range(n_samples):
      Q_k = np.array([feat_map.map(next_states[i]).dot(theta_old[:,a]) for a in range(env.Na)] )
      f = max(Q_k)       
      #clipping 
      if f > 0 :
        f = min(f , R_max / (1-gamma) )
      else : 
        f = max(f, - R_max/(1-gamma))

      y_k[i] = rewards[i] + gamma* f


    ## calcul de theta_{k+1} : theta_new
    
    theta_new = np.zeros((feat_map.dim, env.Na))
    for a in range(env.Na):
      index_list = [i for i, x in enumerate(actions) if x == a]

      if len(index_list) != 0:
        mat_to_inv = sum([np.matmul(np.array([feat_map.map(states[i])]).T, np.array([feat_map.map(states[i])])) for i in index_list]) + lambd*np.eye(feat_map.dim)
        mat_sum = sum([y_k[i]*feat_map.map(states[i]) for i in index_list])
        theta_new[:,a] = np.matmul(np.linalg.inv(mat_to_inv),mat_sum)

  return theta_new


In [ ]:

# ----------------------------
# Environment and feature map
# ----------------------------
env = get_large_gridworld()
# you can change the parameters of the feature map, and even try other maps!
feat_map = GridWorldFeatureMap(env, dim=50, sigma=0.25)

# -------
# Run FQI
# -------
theta = linear_fqi(env, feat_map, num_iterations= 100)

# Compute and run greedy policy
Q_fqi = np.zeros((env.Ns, env.Na))
for ss in range(env.Ns):
  state_feat = feat_map.map(ss)
  Q_fqi[ss, :] = state_feat @ theta

V_fqi = Q_fqi.max(axis=1)
policy_fqi = Q_fqi.argmax(axis=1)
render_policy(env, policy_fqi, horizon=100)

# Visualize the approximate value function in the gridworld.
img = env.get_layout_img(V_fqi)    
plt.imshow(img)
plt.show()

**Response**


 To reduce the approximation error:
 
$\rightarrow$ I clipped $f$ ie when $max Q_k > \frac{Rmax}{1-\gamma}$ (resp. $< -\frac{Rmax}{1-\gamma}$), I used $\frac{Rmax}{1-\gamma}$ (resp. $ -\frac{Rmax}{1-\gamma}$) to compute $y^i_k$ 


$\rightarrow$ I used the uniformly sampling method to collect the dataset, because we saw part 2, this method gave better results for the estimation of Q and V.

In [ ]:
## Comparaison of the results with Value Iteration

# Parameters
tol = 1e-3
gamma = 0.9

VI_Q, VI_greedypol, all_qfunctions, VI_nb_it, VI_list_time_it = value_iteration(env.P, env.R, gamma=gamma, tol=tol)


mae_FQI = mean_absolute_error(VI_Q, Q_fqi)
print("MAE for FQI method :",mae_FQI)
mse_FQI = mean_squared_error(VI_Q, Q_fqi)
print("MSE for FQI method :",mse_FQI)


In [ ]:
render_policy(env, VI_greedypol, horizon=100)
img = env.get_layout_img(VI_Q.max(axis=1))    
plt.imshow(img)
plt.show()

Both policies lead the red diamond to its objectif, but we can see on the map that true V is higher than estimated V with FQI.

In [ ]:
## Compute Q_fqi for diffenrent sigma and dim

list_sigma = [0.1, 0.3, 0.6, 0.9]
list_dim = [10,30,50, 80]

list_param_1 =[]
list_param_2 =[]
list_Q_fqi = []
for dim in list_dim :
  for sigma in list_sigma:
    print("feat map with sigma =",sigma, 'and dim = ', dim)
    list_param_1.append(dim)
    list_param_2.append(sigma)
    
    feat_map = GridWorldFeatureMap(env, dim=50, sigma=0.25)

    theta = linear_fqi(env, feat_map, num_iterations= 100)

    # Compute and run greedy policy
    Q_fqi = np.zeros((env.Ns, env.Na))
    for ss in range(env.Ns):
      state_feat = feat_map.map(ss)
      Q_fqi[ss, :] = state_feat @ theta

    list_Q_fqi.append(Q_fqi)

In [ ]:
list_mae = []
for q in list_Q_fqi:
 list_mae.append(mean_absolute_error(VI_Q, q))

In [ ]:
import pandas as pd

df = pd.DataFrame(list(zip(list_param_1, list_param_2, list_mae)), columns =["Dim","Sigma","MAE"])
plt.scatter(df.Dim, df.Sigma, c=df.MAE)
plt.colorbar()
plt.xlabel("Dim")
plt.ylabel("Sigma")
plt.title("MAE according to dim and sigma of feat map")

It seems that the estimation of Q with FQI is better with bigger dimension of features map (>50) and with sigma around 0.5.